In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
from matplotlib import cm
from test_bench import *
import itertools
%matplotlib inline
os.environ['OMP_NUM_THREADS'] = '6'
print (os.environ['OMP_NUM_THREADS'])

6


In [2]:
num_expr = 1 # Adding this num to output fn for distinguish between experiments  (not used for now)
nder = 2 # Dimension
ToLoadSet = False # whether to load settigns form file

functions = [f_rosenbrock, f_sincos, f_roots, f_gauss]
inital_points_distribs = ['random', 'LHS']

if ToLoadSet:
    from sandbox_bm_approx_settings import *
else:
    dir_str = os.path.join(os.environ['HOME'], 'work/res/bm-big/')
    num_points_for_big_matrix = 10000 # number of points for big matrix
    max_row =  30                    # Maximum number of points taken in experiments
    max_expansion = 30              # number of columns in big matrix in (nder+1) units
    min_expansion = 2                  # minimal number of columns in experiments in (nder+1) units
    

    n_test = 50000    # points on test grid (for calculating error on final step)
    poly = cheb       # used polinomials


#####
dir_pdf = os.path.join(dir_str, "pdf")
try:
    os.makedirs(dir_pdf)
except:
    pass



ToTakePointsFromFile = False # (not used for now)

# ---------------------------------
p_size = (nder+1)*max_row #number of rows in big matrix

### generating test points
points_test = test_points_gen(n_test, nder, distrib='random') 


#contents of file sandbox_bm_approx_settings.py
_="""
from gen_mat import cheb


dir_str = '.'
num_points_for_big_matrix = 50000 # number of points for big matrix
max_row = 50                      # Maximum number of points taken in experiments
max_expansion = 10                 # number of columns in big matrix in (nder+1) units
min_expansion = 1                 # minimal number of columns in experiments in (nder+1) units

n_test = 50000    # points on test grid (for calculating error on final step)
poly = cheb       # used polynomials
"""

### Testing the new most general environment

In [ ]:
### evaluating test
for inital_points_distrib in inital_points_distribs:
    points_fn = 'taken_points_{}'.format(inital_points_distrib)
    x = test_points_gen(num_points_for_big_matrix, nder, distrib=inital_points_distrib)
    
    A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    
    np.savez(os.path.join(dir_str, points_fn), x=x, points_test=points_test)
    
    fn_pre = lambda function : "func={}_poly={}_distrib={}".format(function.__name__, poly.__name__, inital_points_distrib)
    fls = [open(os.path.join(dir_str, fn_pre(function) + '.txt'), "w") for function in functions]
    
    fn_pre_pdf = "poly={}_distrib={}".format(poly.__name__, inital_points_distrib)
    for expansion in range(min_expansion, max_expansion):
                for N_rows_ex in range(max_row, expansion, -1): # It's not the way people do...
                    N_rows = N_rows_ex*(nder+1)
                    fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(expansion, N_rows_ex))
                    try:
                        error, taken_points = test(A, x, points_test, nder, expansion, N_rows, functions, 
                                                   poly=poly, to_save_pivs=N_rows_ex==max_row, 
                                                   fnpdf=fnpdf)
                    except SingularError as err:
                        print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                                            expansion, N_rows_ex, err.value)) 
                        #continue
                        break

                    for i, f in enumerate(fls):
                        taken_points.tofile(f, sep=" ")
                        f.write("_error={}_Nrows={}_expans={}\n".format(error[i], N_rows, expansion))
                        f.flush()
                    
    for f in fls:
        f.close()


not full column rank with expansion=2, N_rows_ex=30, err=2
not full column rank with expansion=5, N_rows_ex=30, err=2


In [ ]:
def file_extraction(Filepath):
    srch = re.compile(r'([\d\s]+)_error=([\+\-\d\.eE]+)_Nrows=(\d+)_expans=(\d+)')
    fnd = srch.findall(open(Filepath, 'r').read())
    return tuple(np.array(i) for i in zip(*[(float(i0), int(i1), int(i2), [int(p) for p in im1.strip().split(' ') if len(p) > 0])
                                            for im1, i0, i1, i2 in fnd]))


def DataToMesh(error, N_row, N_col, *args):
    row_s = sorted(list(set(N_row)))
    col_s = sorted(list(set(N_col)))
    data = {(N_row[i], N_col[i]) : e for i, e in enumerate(error)}
    
    res = np.empty((len(row_s), len(col_s)), dtype=float)
    for i, r in enumerate(row_s):
        for j, c in enumerate(col_s):
            try:
                res[i,j] = data[(r, c)]
            except:
                res[i,j] = np.nan
    X, Y = np.meshgrid(row_s, col_s)
    return res.T, X, Y

def PlotError(fn, log_it=False):
    error, N_row, N_col = DataToMesh(*file_extraction(fn))
    if log_it:
        error = np.log10(error)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(N_row, N_col, error, edgecolor='black', linewidth=0.5, cmap = cm.Spectral)
    # ax.legend()
    ax.set_xlabel('N_rows', fontsize=10)
    ax.set_ylabel('N_columns')
    plt.show()
    
    
# Not used
def PlotPoints(fn, row, col, x, to_save_fig=False, fn_out="func.pdf"):
    _, N_row, N_col, idx = file_extraction(fn)
    
    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = (u_bound - l_bound)/20.0
    plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
    plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
    plt.plot(x[idx, 0], x[idx, 1], 'b^')
    # plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=2, borderaxespad=0.)
    # plt.title("E = {}".format(error))
    plt.grid(True)
    if to_save_fig:
        # fn = 'func={}_d={}_num={}_nder={}.pdf'.format(function.__name__, N_column, N_rows, nder)
        plt.savefig(fn_out)
    

In [ ]:
PlotError("func=Rosenbrock_poly=cheb.txt", True)

## Test PLUQ

In [ ]:
def vol2(A):
    return(np.sqrt(np.abs(la.det(np.dot(A.T, A)))))

In [ ]:
inital_points_distrib = 'LHS'
nder = 2
row_exp = max_row + 4
del_dist = 0.2

num_points_for_big_matrix = 3600
poly = cheb
Kmax = row_exp*(nder+1)
x = test_points_gen(num_points_for_big_matrix, nder, distrib=inital_points_distrib)
    
A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
A = matrix_prep(A, nder+1)

In [ ]:
print A.shape

In [ ]:
test = np.load("taken_points_random.npz")

In [ ]:
p,q,lu,indx = ids.pluq_ids(A,nder,do_pullback=False,pullbacks=4,overwrite_a=False,preserve_order=True)
A = A[p]

In [ ]:
A1 = np.copy(A)
erase_init(point_erase, x, nder, del_dist)
pivs = rect_block_maxvol(A1, nder, Kmax = Kmax, max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, to_erase = point_erase)

In [ ]:
cut_piv = pivs
taken_indices = cut_piv[::(nder+1)] // (nder+1)
l_bound = np.amin(x, 0)
u_bound = np.amax(x, 0)
delta = (u_bound - l_bound)/20.0
fig = plt.figure(figsize=(10,10))
plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
plt.plot(x[taken_indices, 0], x[taken_indices, 1], 'b^')
# plt.title("E = {}".format(error))
plt.grid(True)

In [ ]:
cut_piv = pivs[:Kmax]
taken_indices = cut_piv[::(nder+1)] // (nder+1)
l_bound = np.amin(x, 0)
u_bound = np.amax(x, 0)
delta = (u_bound - l_bound)/20.0
fig = plt.figure()
plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
plt.plot(x[taken_indices, 0], x[taken_indices, 1], 'b^')
# plt.title("E = {}".format(error))
plt.grid(True)

In [ ]:
from numba import jit

#@jit
def AllCombsrect(A, l, num_take_blocks=None, debug=False):
    # INPUT 
    # A -- matrix
    # l -- number of dimensions (derivatives), size of block is l+1
    # num_take_bocks -- blocks returnd by algorithm
    l1 = l + 1
    b_one = np.arange(l1)
    num_of_blocks = A.shape[0] // l1
    max_det = 0.0
    good_idx = None
    if num_take_blocks is None:
        num_take_blocks = A.shape[1] // l1 # Take square mat and test square block maxvol
    for i in itertools.combinations(xrange(num_of_blocks), num_take_blocks):
        # print i
        idx = np.hstack((b_one + j*l1 for j in i))
        Al = A[idx]
        B  = np.dot(Al.T,Al)
        cur_det = np.abs(la.det(B))
        if cur_det > max_det:
            max_det = cur_det
            good_idx = i
        if debug:
            print i, idx, cur_det
            
    return max_det, good_idx

In [ ]:
def GetNumOfBlocks(idx, l):
    l1 = l+1
    b_one = np.arange(l1)
    idx_bl = idx[np.arange(0, len(idx), l1)] // (l1)
    assert( np.all(np.hstack((b_one + j*l1 for j in idx_bl)) == idx)  )
    return idx_bl

In [ ]:
A = GenMat((nder+1), x, poly=poly, debug=False, pow_p=1)
A = matrix_prep(A, nder+1)

In [ ]:
det, indices_best = AllCombsrect(A, nder, num_take_blocks=6, debug=False)
np.save('indices', indices_best)
np.save('supervol', det)

In [ ]:
indices_best = np.array(indices_best)

In [ ]:
piv_max = np.array([27,28,29,39,40,41,54,55,56,60,61,62,153,154,155,177,178,179])

In [ ]:
block_func_deriv = RHS(sincos, x[indices_best])
c_block3, res_x, rank, s = np.linalg.lstsq(A[piv_max], block_func_deriv)
    
approx_calcul = approximant(nder, c_block3, poly=poly)
error3 = error_est(sincos, approx_calcul, points_test)

In [ ]:
def printing(x,taken_indices,N_column,N_rows):
    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = 0.15
    fig = plt.figure()
    plt.xlim(l_bound[0] - delta, u_bound[0] + delta)
    plt.ylim(l_bound[1] - delta, u_bound[1] + delta)
    plt.plot(x[taken_indices, 0], x[taken_indices, 1], 'b^')
    # plt.title("E = {}".format(error))
    plt.grid(True)
    
    # fnpdf = 'func={}_columns={}_rows={}_pnts={}.pdf'.format(function.__name__, N_column, N_rows, len(taken_indices))
    fnpdf = 'columns={}_rows12={}.pdf'.format(N_column, N_rows)
    # print "Num of points = {}, saving to file {}".format(len(taken_indices), fnpdf)
    plt.savefig(fnpdf)
    plt.close(fig)